This Jupyter notebook loads the raw amplitude and Mel spectrogram data files as numpy arrays.

Download the data files [here](https://console.cloud.google.com/storage/browser/cs181_practical_data).  This notebook assumes that the data files as located in the same directory.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

### Load raw amplitude data

In [ ]:
# Load train data
X_amp_train = np.load("Xtrain_amp.npy")
y_amp_train = np.load("ytrain_amp.npy")

In [ ]:
X_amp_train.shape

In [ ]:
y_amp_train
sns.countplot(y_amp_train)

In [ ]:
# Load test data
X_amp_test = np.load("Xtest_amp.npy")
y_amp_test = np.load("ytest_amp.npy")

In [ ]:
X_amp_test.shape

### Load Mel spectrogram data

In [ ]:
# Load train data
X_mel_train = np.load("Xtrain_mel.npy")
y_mel_train = np.load("ytrain_mel.npy")

In [ ]:
X_mel_train.shape

In [ ]:
# Flatten X_mel_train's spectrogram features
X_mel_train_flat = X_mel_train.reshape(X_mel_train.shape[0], -1)
X_mel_train_flat.shape

In [ ]:
# Load test data
X_mel_test = np.load("Xtest_mel.npy")
y_mel_test = np.load("ytest_mel.npy")

In [ ]:
X_mel_test.shape

In [ ]:
# Flatten X_mel_test's spectrogram features
X_mel_test_flat = X_mel_test.reshape(X_mel_test.shape[0], -1)
X_mel_test_flat.shape

### Part A: Feature Engineering and Baseline Models

**Your task: Train the following two models.**

1. Perform PCA on the raw amplitude features (`Xtrain amp`, `Xtest amp`). Train a logistic regression model on the 500 most significant PCA components. This will be our first baseline model.

2. Perform PCA on the Mel spectrogram features (`Xtrain mel`, `Xtest mel`). Train a logistic regression model on the 500 most significant PCA components. This will be our second baseline model.

**Discuss which feature representation resulted in higher model performance, and why you hypothesize this feature representation performed better than the other. Also discuss why we might have asked you to perform PCA first and the impact of that choice.**

In [ ]:
# run PCA for everything
pca = PCA(n_components=500)

X_amp_train_pca = pca.fit_transform(X_amp_train)
X_amp_test_pca = pca.transform(X_amp_test)

X_mel_train_flat_pca = pca.fit_transform(X_mel_train_flat)
X_mel_test_flat_pca = pca.transform(X_mel_test_flat)

In [ ]:
my_model = PCA(n_components=500)
my_model.fit_transform(X_amp_train)

print(my_model.explained_variance_ratio_.cumsum()[499])

plt.plot(np.arange(1, 501), my_model.explained_variance_ratio_.cumsum())
plt.xlabel("Number of Components")
plt.ylabel("Proportion of Variance Explained")
plt.title("PCA for Amp Feature Representation")
plt.savefig('pca_amp.png')

In [ ]:
my_model = PCA(n_components=500)
my_model.fit_transform(X_mel_train_flat)

print(my_model.explained_variance_ratio_.cumsum()[499])

plt.plot(np.arange(1, 501), my_model.explained_variance_ratio_.cumsum())
plt.xlabel("Number of Components")
plt.ylabel("Proportion of Variance Explained")
plt.title("PCA for Mel Feature Representation")
plt.savefig('pca_mel.png')

In [ ]:
# First Baseline Model

train_accuracies = []
test_accuracies = []
train_per_class_accuracies = []
test_per_class_accuracies = []

# create our PCA object that will calculate the first 500 principal components
pca = PCA(n_components=500)

# fit and transform our training data
X_amp_train_pca = pca.fit_transform(X_amp_train)

# train a logistic regression on the PCA transformed training data
# max_iter=1000 to get rid of covergence warnings
lr = LogisticRegression(max_iter=1000)
lr.fit(X_amp_train_pca, y_amp_train)

# make our train predictions
preds_amp_train_lr = lr.predict(X_amp_train_pca)

# transform our testing data
X_amp_test_pca = pca.transform(X_amp_test)

# make our test predictions
preds_amp_test_lr = lr.predict(X_amp_test_pca)

# calculate train and test accuracy
train_accuracies.append(accuracy_score(y_amp_train, preds_amp_train_lr))
test_accuracies.append(accuracy_score(y_amp_test, preds_amp_test_lr))

# confusion matrix
train_cfm = confusion_matrix(y_amp_train, preds_amp_train_lr)
test_cfm = confusion_matrix(y_amp_test, preds_amp_test_lr)

# calculate per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
train_per_class_accuracies.append(train_cfm.diagonal() / train_cfm.sum(axis=1))
test_per_class_accuracies.append(test_cfm.diagonal() / test_cfm.sum(axis=1))

print(f"TRAIN ACCURACY: {np.mean(train_accuracies)}")
print(f"TEST ACCURACY: {np.mean(test_accuracies)}")
print(f"TRAIN PER-CLASS ACCURACY: {np.mean(train_per_class_accuracies, axis=0)}")
print(f"TEST PER-CLASS ACCURACY: {np.mean(test_per_class_accuracies, axis=0)}")

In [ ]:
# Second Baseline Model

train_accuracies = []
test_accuracies = []
train_per_class_accuracies = []
test_per_class_accuracies = []

# train a logistic regression on the PCA transformed training data
# solver='sag' and max_iter=10000 to get rid of covergence warnings
lr_pca = LogisticRegression(solver='sag', max_iter=10000)
lr_pca.fit(X_mel_train_flat_pca, y_mel_train)

# make our train predictions
preds_mel_train_lr_pca = lr_pca.predict(X_mel_train_flat_pca)

# make our test predictions
preds_mel_test_lr_pca = lr_pca.predict(X_mel_test_flat_pca)

# calculate train and test accuracy
train_accuracies.append(accuracy_score(y_mel_train, preds_mel_train_lr_pca))
test_accuracies.append(accuracy_score(y_mel_test, preds_mel_test_lr_pca))

# confusion matrix
train_cfm = confusion_matrix(y_mel_train, preds_mel_train_lr_pca)
test_cfm = confusion_matrix(y_mel_test, preds_mel_test_lr_pca)

# get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
train_per_class_accuracies.append(train_cfm.diagonal() / train_cfm.sum(axis=1))
test_per_class_accuracies.append(test_cfm.diagonal() / test_cfm.sum(axis=1))

print(f"TRAIN ACCURACY: {np.mean(train_accuracies)}")
print(f"TEST ACCURACY: {np.mean(test_accuracies)}")
print(f"TRAIN PER-CLASS ACCURACY: {np.mean(train_per_class_accuracies, axis=0)}")
print(f"TEST PER-CLASS ACCURACY: {np.mean(test_per_class_accuracies, axis=0)}")

### Part B: More Modeling

Now, you will experiment with more expressive nonlinear model classes to maximize accuracy on the audio classification task. Examples of nonlinear models include random forests, KNN, and neural networks.

**B1: First Step**

First, we will be looking at simple models that are slightly more complicated than a linear
model.

**Your task: Train at least one nonlinear model on a feature representation of your choice. For model classes with hyperparameters, select a hyperparameter value you intuitively think is appropriate. Compare your results to the logistic regression models in Part A and discuss what your results imply about the task.**

In [ ]:
# amp Random Forest

train_accuracies = []
test_accuracies = []
train_per_class_accuracies = []
test_per_class_accuracies = []

for i in range(5):

    # set seed
    np.random.seed(100 + 50 * i)

    # train a random forest on the PCA transformed training data
    model = RandomForestClassifier(max_depth=math.log(X_mel_train.shape[0]), n_jobs=-1)
    model.fit(X_amp_train_pca, y_amp_train)

    # make our train predictions
    preds_amp_train_rf = model.predict(X_amp_train_pca)

    # make our test predictions
    preds_amp_test_rf = model.predict(X_amp_test_pca)

    # calculate train and test accuracy
    train_accuracies.append(accuracy_score(y_amp_train, preds_amp_train_rf))
    test_accuracies.append(accuracy_score(y_amp_test, preds_amp_test_rf))

    # confusion matrix
    train_cfm = confusion_matrix(y_amp_train, preds_amp_train_rf)
    test_cfm = confusion_matrix(y_amp_test, preds_amp_test_rf)

    # get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
    train_per_class_accuracies.append(train_cfm.diagonal() / train_cfm.sum(axis=1))
    test_per_class_accuracies.append(test_cfm.diagonal() / test_cfm.sum(axis=1))

print(f"TRAIN ACCURACY: {np.mean(train_accuracies)}")
print(f"TEST ACCURACY: {np.mean(test_accuracies)}")
print(f"TRAIN PER-CLASS ACCURACY: {np.mean(train_per_class_accuracies, axis=0)}")
print(f"TEST PER-CLASS ACCURACY: {np.mean(test_per_class_accuracies, axis=0)}")

In [ ]:
# mel Random Forest

train_accuracies = []
test_accuracies = []
train_per_class_accuracies = []
test_per_class_accuracies = []

for i in range(5):

    # set seed
    np.random.seed(100 + 50 * i)

    # train a random forest on the PCA transformed training data
    model = RandomForestClassifier(max_depth=math.log(X_mel_train.shape[0]), n_jobs=-1)
    model.fit(X_mel_train_flat_pca, y_mel_train)

    # make our train predictions
    preds_mel_train_rf = model.predict(X_mel_train_flat_pca)

    # make our test predictions
    preds_mel_test_rf = model.predict(X_mel_test_flat_pca)

    # calculate train and test accuracy
    train_accuracies.append(accuracy_score(y_mel_train, preds_mel_train_rf))
    test_accuracies.append(accuracy_score(y_mel_test, preds_mel_test_rf))

    # confusion matrix
    train_cfm = confusion_matrix(y_mel_train, preds_mel_train_rf)
    test_cfm = confusion_matrix(y_mel_test, preds_mel_test_rf)

    # get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
    train_per_class_accuracies.append(train_cfm.diagonal() / train_cfm.sum(axis=1))
    test_per_class_accuracies.append(test_cfm.diagonal() / test_cfm.sum(axis=1))

print(f"TRAIN ACCURACY: {np.mean(train_accuracies)}")
print(f"TEST ACCURACY: {np.mean(test_accuracies)}")
print(f"TRAIN PER-CLASS ACCURACY: {np.mean(train_per_class_accuracies, axis=0)}")
print(f"TEST PER-CLASS ACCURACY: {np.mean(test_per_class_accuracies, axis=0)}")

In [ ]:
# amp KNN

model = KNeighborsClassifier(n_neighbors=10)
model.fit(X_amp_train_pca, y_amp_train)

# make our train predictions
preds_amp_train_knn = model.predict(X_amp_train_pca)

# make our test predictions
preds_amp_test_knn = model.predict(X_amp_test_pca)

# calculate train and test accuracy
train_accuracy = accuracy_score(y_amp_train, preds_amp_train_knn)
test_accuracy = accuracy_score(y_amp_test, preds_amp_test_knn)

# confusion matrix
train_cfm = confusion_matrix(y_amp_train, preds_amp_train_knn)
test_cfm = confusion_matrix(y_amp_test, preds_amp_test_knn)

# get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
train_per_class_accuracies = train_cfm.diagonal() / train_cfm.sum(axis=1)
test_per_class_accuracies = test_cfm.diagonal() / test_cfm.sum(axis=1)

print(f"TRAIN ACCURACY: {train_accuracy}")
print(f"TEST ACCURACY: {test_accuracy}")
print(f"TRAIN PER-CLASS ACCURACY: {train_per_class_accuracies}")
print(f"TEST PER-CLASS ACCURACY: {test_per_class_accuracies}")

In [ ]:
# mel KNN

# train a KNN model on the PCA transformed training data
model = KNeighborsClassifier(n_neighbors=10)
model.fit(X_mel_train_flat_pca, y_mel_train)

# make our train predictions
preds_mel_train_knn = model.predict(X_mel_train_flat_pca)

# make our test predictions
preds_mel_test_knn = model.predict(X_mel_test_flat_pca)

# calculate train and test accuracy
train_accuracy = accuracy_score(y_mel_train, preds_mel_train_knn)
test_accuracy = accuracy_score(y_mel_test, preds_mel_test_knn)

# confusion matrix
train_cfm = confusion_matrix(y_mel_train, preds_mel_train_knn)
test_cfm = confusion_matrix(y_mel_test, preds_mel_test_knn)

# get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
train_per_class_accuracies = train_cfm.diagonal() / train_cfm.sum(axis=1)
test_per_class_accuracies = test_cfm.diagonal() / test_cfm.sum(axis=1)

print(f"TRAIN ACCURACY: {train_accuracy}")
print(f"TEST ACCURACY: {test_accuracy}")
print(f"TRAIN PER-CLASS ACCURACY: {train_per_class_accuracies}")
print(f"TEST PER-CLASS ACCURACY: {test_per_class_accuracies}")

**B2: More Complicated Models–Hyperparameter Tuning and Validation**

In this section, you will explore hyperparameter tuning. Model hyperparameters such as network architecture or random forest maximum tree depth determine the expressivity of the model class. Training hyperparameters such as learning rate, weight decay, or regularization coefficients influence optimization and can encourage desirable properties (such as sparsity) in the final learned models.

Popular hyperparameter tuning techniques include random search, where you train a set of models with hyperparameters chosen uniformly at random from a set of possible values, and grid search, where all possible parameter values are considered exhaustively.

**Your task: Perform a hyperparameter search to maximize predictive accuracy for two model classes of your choice. You can choose which hyperparameters you search over (feel free to search over multiple simultaneously if you’d like!), but you must search over at least 5 possible values for at least 1 hyperparameter. Explore the changes in performance as you choose different hyperparameter values. In your writeup, discuss your validation strategy and your conclusions.**

Note: Choose how to present your results of your hyperparameter search in a way that best reflects how to communicate your conclusions.

In [ ]:
# relevant hyperparameters:
# n_estimators = number of trees in the foreset
# min_samples_split = min number of data points placed in a node before the node is
# max_depth = max number of levels in each decision tree

n_estimators_search = [50, 100, 150, 200, 250]
min_samples_split_search = [2, 3, 4, 5, 6]
max_depth_search = [3, 8, 13, 18, None]

grid_search = {'n_estimators': n_estimators_search,
               'min_samples_split': min_samples_split_search,
               'max_depth': max_depth_search}

In [ ]:
# instantiate our gridsearch estimator - cv=None defaults to the 5-fold cross validation
RFC = RandomForestClassifier(random_state=181)
RFC_CV = GridSearchCV(estimator=RFC, param_grid=grid_search, n_jobs=-1, cv=None, verbose=1)

RFC_CV.fit(X_amp_train_pca, y_amp_train)

# select our best model from the gridsearch
best_params = RFC_CV.best_params_
print(best_params)

train_accuracies = []
test_accuracies = []
train_per_class_accuracies = []
test_per_class_accuracies = []

for i in range(5):
    # set seed
    np.random.seed(100 + 50 * i)
    RFC_best = RandomForestClassifier(n_estimators=best_params['n_estimators'], min_samples_split=best_params['min_samples_split'], max_depth=best_params['max_depth'],
                                     random_state = 181)

    # fit it on our entire train data
    RFC_best.fit(X_amp_train_pca, y_amp_train)
    
    preds_best_amp_train_rf = RFC_best.predict(X_amp_train_pca)
    preds_best_amp_test_rf = RFC_best.predict(X_amp_test_pca)
    
    # train and test accuracy
    train_accuracies.append(accuracy_score(y_amp_train, preds_best_amp_train_rf))
    test_accuracies.append(accuracy_score(y_amp_test, preds_best_amp_test_rf))

    # confusion matrix
    train_cfm = confusion_matrix(y_amp_train, preds_best_amp_train_rf)
    test_cfm = confusion_matrix(y_mel_test, preds_best_amp_test_rf)

    # get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
    train_per_class_accuracies.append(train_cfm.diagonal() / train_cfm.sum(axis=1))
    test_per_class_accuracies.append(test_cfm.diagonal() / test_cfm.sum(axis=1))

print(f"TRAIN ACCURACY: {np.mean(train_accuracies)}")
print(f"TEST ACCURACY: {np.mean(test_accuracies)}")
print(f"TRAIN PER-CLASS ACCURACY: {np.mean(train_per_class_accuracies, axis=0)}")
print(f"TEST PER-CLASS ACCURACY: {np.mean(test_per_class_accuracies, axis=0)}")

In [ ]:
# instantiate our gridsearch estimator - cv=None defaults to the 5-fold cross validation
RFC = RandomForestClassifier(random_state=181)
RFC_CV = GridSearchCV(estimator=RFC, param_grid=grid_search, n_jobs=-1, cv=None, verbose=1)

RFC_CV.fit(X_mel_train_flat_pca, y_mel_train)

# select our best model from the gridsearch
best_params = RFC_CV.best_params_
print(best_params)

train_accuracies = []
test_accuracies = []
train_per_class_accuracies = []
test_per_class_accuracies = []

for i in range(5):
    # set seed
    np.random.seed(100 + 50 * i)
    RFC_best = RandomForestClassifier(n_estimators=best_params['n_estimators'], min_samples_split=best_params['min_samples_split'], max_depth=best_params['max_depth'],
                                     random_state = 181)

    # fit it on our entire train data
    RFC_best.fit(X_mel_train_flat_pca, y_mel_train)
    
    preds_best_mel_train_rf = RFC_best.predict(X_mel_train_flat_pca)
    preds_best_mel_test_rf = RFC_best.predict(X_mel_test_flat_pca)
    
    # train and test accuracy
    train_accuracies.append(accuracy_score(y_mel_train, preds_best_mel_train_rf))
    test_accuracies.append(accuracy_score(y_mel_test, preds_best_mel_test_rf))

    # confusion matrix
    train_cfm = confusion_matrix(y_mel_train, preds_best_mel_train_rf)
    test_cfm = confusion_matrix(y_mel_test, preds_best_mel_test_rf)

    # get per-class classification accuracy (https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class)
    train_per_class_accuracies.append(train_cfm.diagonal() / train_cfm.sum(axis=1))
    test_per_class_accuracies.append(test_cfm.diagonal() / test_cfm.sum(axis=1))

print(f"TRAIN ACCURACY: {np.mean(train_accuracies)}")
print(f"TEST ACCURACY: {np.mean(test_accuracies)}")
print(f"TRAIN PER-CLASS ACCURACY: {np.mean(train_per_class_accuracies, axis=0)}")
print(f"TEST PER-CLASS ACCURACY: {np.mean(test_per_class_accuracies, axis=0)}")

In [ ]:
# amp KNN Grid Search

# these are the settings that we will tune: 'n_neighbors'
param_grid = {'n_neighbors': np.arange(1, 28, 2),}

# instantiate our template model
KNN = KNeighborsClassifier()

# instantiate our gridsearch estimator - cv=None defaults to the 5-fold cross validation
KNN_CV = GridSearchCV(estimator=KNN, param_grid=param_grid, n_jobs=-1, cv=None, verbose=1)
KNN_CV.fit(X_amp_train_pca, y_amp_train)

# print the best results
print(KNN_CV.best_params_)

# select our best model from the gridsearch
best_params = KNN_CV.best_params_
KNN_best = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])

# fit it on our entire train data
KNN_best.fit(X_amp_train, y_amp_train)

# make our train predictions
preds_amp_train_knn_best = KNN_best.predict(X_amp_train)

# make our test predictions
preds_amp_test_knn_best = KNN_best.predict(X_amp_test)

train_cfm = confusion_matrix(y_amp_train, preds_amp_train_knn_best)
test_cfm = confusion_matrix(y_amp_test, preds_amp_test_knn_best)

print(f"TRAIN ACCURACY: {accuracy_score(y_amp_train, preds_amp_train_knn_best)}")
print(f"TEST ACCURACY: {accuracy_score(y_amp_test, preds_amp_test_knn_best)}")
print(f"TRAIN PER-CLASS ACCURACY: {train_cfm.diagonal() / train_cfm.sum(axis=1)}")
print(f"TEST PER-CLASS ACCURACY: {test_cfm.diagonal() / test_cfm.sum(axis=1)}")

In [ ]:
#### mel KNN Grid Search

# these are the settings that we will tune: 'n_neighbors'
param_grid = {'n_neighbors': np.arange(1, 28, 2),}

# instantiate our template model
KNN = KNeighborsClassifier()

# instantiate our gridsearch estimator - cv=None defaults to the 5-fold cross validation
KNN_CV = GridSearchCV(estimator=KNN, param_grid=param_grid, n_jobs=-1, cv=None, verbose=1)
KNN_CV.fit(X_mel_train_flat_pca, y_mel_train)

# print the best results
print(KNN_CV.best_params_)

# select our best model from the gridsearch
best_params = KNN_CV.best_params_
KNN_best = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])

# fit it on our entire train data
KNN_best.fit(X_mel_train_flat_pca, y_mel_train)

# make our train predictions
preds_mel_train_knn_best = KNN_best.predict(X_mel_train_flat_pca)

# make our test predictions
preds_mel_test_knn_best = KNN_best.predict(X_mel_test_flat_pca)

train_cfm = confusion_matrix(y_mel_train, preds_mel_train_knn_best)
test_cfm = confusion_matrix(y_mel_test, preds_mel_test_knn_best)

print(f"TRAIN ACCURACY: {accuracy_score(y_mel_train, preds_mel_train_knn_best)}")
print(f"TEST ACCURACY: {accuracy_score(y_mel_test, preds_mel_test_knn_best)}")
print(f"TRAIN PER-CLASS ACCURACY: {train_cfm.diagonal() / train_cfm.sum(axis=1)}")
print(f"TEST PER-CLASS ACCURACY: {test_cfm.diagonal() / test_cfm.sum(axis=1)}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1b5e2bb3-566b-4513-989d-4151b916dec0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>